# Try HillPositive in BioCRNPyler
<font color = red>
    OK LOL this was a bad idea because MM kinetics and hill functions can only be used for single substrate reactions:((
    </font> <br>
Same reduced model? Have KM and Kcat from paper<br>
7.5.2020

We will use bioscrape positive hill function. For parameters, we will use 'rate' as k_cat, which are indicated as enzyme activity in the paper supplement. Then, we will estimate K_M for each enzyme from online source.<br>
We should use bioscrape 'proportionalhillpositive' since the extra term will be the total concentration of the enzyme

In [1]:
e0 = 1.5
str(e0)

'1.5'

In [ ]:
e0 = 0.15
rxn1 = (["glucose", 'atp'], ["g6p"], "proportionalhillpositive", {"d":str(e0), "s1":"glucose", "k":"ktx", "K":"KI", "n":"n"})




In [ ]:
E1_hex = Enzyme(enzyme_name = "hex", substrate = ['glucose'],
                fuel = ['atp'],product = ['g6p'], waste = ['adp'], k_bf = k_bf , k_uf = k_uf)

    E2_pgi = Enzyme(enzyme_name = 'pgi', substrate = ['g6p'], fuel = [],
               product = ['f6p'], waste = [], k_bf = k_bf , k_uf = k_uf)

    E3_pfk = Enzyme(enzyme_name = 'pfk', substrate = ['f6p'], fuel = ['atp'], product = ['f16p'],
               waste = ['adp'], k_bf = k_bf , k_uf = k_uf)

    E4_ald_tpi = Enzyme(enzyme_name ='ald_tpi' , substrate = ['f16p'], fuel = [], product = ['g3p', 'g3p'], 
                waste = [], k_bf = k_bf , k_uf = k_uf )

    E5_gapN = Enzyme(enzyme_name ='gapN' , substrate = ['g3p', 'g3p'], fuel = ['nadp', 'nadp'], product = ['3pg', '3pg'], 
                waste = ['nadph', 'nadph'], k_bf = k_bf , k_uf = k_uf)

    E6_gapM6 = Enzyme(enzyme_name ='gapM6' , substrate = ['g3p', 'g3p'], fuel = ['pi'], product = ['13bpg'], 
                waste = [],k_bf = k_bf , k_uf = k_uf)

    E7_pgk = Enzyme(enzyme_name = 'pgk', substrate = ['13bpg'], fuel = ['adp'], product = ['3pg', '3pg'], 
                waste = ['atp'], k_bf = k_bf , k_uf = k_uf)

    E8_pgm = Enzyme(enzyme_name ='pgm' , substrate = ['3pg', '3pg'], fuel = [], product = ['2pg', '2pg'], 
                waste = [], k_bf = k_bf , k_uf = k_uf)

    E9_eno = Enzyme(enzyme_name ='eno' , substrate = ['2pg', '2pg'], fuel = [], product = ['pep', 'pep'], 
                waste = [],k_bf = k_bf , k_uf = k_uf)

    E10_pyk = Enzyme(enzyme_name = 'pyk', substrate = ['pep', 'pep'], fuel = ['adp', 'adp'], product = ['pyruvate', 'pyruvate'], 
                waste = ['atp', 'atp'], k_bf = k_bf , k_uf = k_uf) # irreversible

    E11_alsS = Enzyme(enzyme_name = 'alsS', substrate = ['pyruvate', 'pyruvate'], fuel = [], product = ['acetolac'], 
                waste = [], k_bf = k_bf , k_uf = k_uf) # irreversible

    E12_IlvC = Enzyme(enzyme_name = 'IlvC', substrate = ['acetolac'], fuel = ['nadph'], product = ['23dih3mebut'], 
                waste = ['nadp'], k_bf = k_bf , k_uf = k_uf)

    E13_IlvD = Enzyme(enzyme_name ='IlvD' , substrate = ['23dih3mebut'], fuel = [], product = ['3me2oxo'], 
                waste = [],k_bf = k_bf , k_uf = k_uf)

    E14_kivD = Enzyme(enzyme_name ='kivD' , substrate = ['3me2oxo'], fuel = [], product = ['isobutanal'], 
                waste = [], k_bf = k_bf , k_uf = k_uf) # irreversible

    E15_yahk = Enzyme(enzyme_name = 'yahk', substrate = ['isobutanal'], fuel = ['nadph'], product = ['isobutanol'],
                      waste = ['nadp'], k_bf = k_bf , k_uf = k_uf)
    
    # Enzyme for ATP Leak
    E16_atpase = Enzyme(enzyme_name = 'atpase', substrate = [], fuel = ['atp'], product = [],
                        waste = ['adp', 'pi'], k_bf = k_bf, k_uf = k_uf, k_cat = k_cat_atp)

In [ ]:
# or something like this from SBTools_BioCRNpyler lecture 2
from biocrnpyler.chemical_reaction_network import Species, Reaction, ChemicalReactionNetwork
delta = 0.1 
K_X = 5
K_S = 20
S = Species(name = "S")
X = Species(name = "X", material_type = "protein")
rxn1 = Reaction([], [X], k = K_X, 
                propensity_type = "hillpositive", 
                propensity_params = {"s1":str(S), "K":K_S, "n":2})

rxn2 = Reaction([X], [], k = delta)
CRN_toy_model = ChemicalReactionNetwork(species = [S,X], reactions = [rxn1, rxn2])
#Species, reactions, and CRNs can all be directly printed
print('CRN for the toy model with non-massaction propensities:\n',CRN_toy_model)

In [ ]:
class FuelMichaelisMenten(Mechanism):
    def __init__(self, name, type = 'catalysis', **keywords):
        
        Mechanism.__init__(self = self, name = name, mechanism_type = type, **keywords)

    def update_species(self, enzyme, fuel_list, substrate_list, product_list, waste_list): 
        
        self.enzyme = check_type(enzyme, 'enzyme')
        
        species = [self.enzyme]
        comp1_list = [self.enzyme]
        comp2_list = [self.enzyme]
        
        for f in fuel_list:
            species.append(f)
            comp1_list.append(f)

        for s in substrate_list:
            species.append(s)
            comp1_list.append(s)
            
        for p in product_list:
            species.append(p)
            comp2_list.append(p)
            
        for w in waste_list:
            species.append(w)
            comp2_list.append(w)
               
        
        species += [ComplexSpecies(comp1_list)]
        species += [ComplexSpecies(comp2_list)]
        return species
    
    def update_reactions(self, enzyme, fuel_list, substrate_list, product_list, waste_list, k_bf, k_uf, k_cat, component = None,
                        part_id = None): 
        
        # Reverse binding rates
        k_br = 0.1*k_bf
        k_ur = 0.1*k_uf

        self.enzyme = check_type(enzyme, 'enzyme')
        
        # Define input lists
        comp1_list = [self.enzyme]
        comp2_list = [self.enzyme]
        
        for f in fuel_list:
            comp1_list.append(f)
            
        for s in substrate_list:
            comp1_list.append(s)

        for p in product_list:
            comp2_list.append(p)
            
        for w in waste_list:
            comp2_list.append(w)
        
        # Define Complexes
        comp1 = ComplexSpecies(comp1_list)
        comp2 = ComplexSpecies(comp2_list)
        
        # Define Reactions
        binding_rxn = Reaction(inputs = comp1_list, outputs=[comp1], k = k_bf, k_rev = k_br)
        cat_rxn = Reaction(inputs = [comp1], outputs = [comp2], k = k_cat)
        unbinding_rxn = Reaction(inputs = [comp2], outputs = comp2_list, k=k_uf, k_rev = k_ur)
        
        return [binding_rxn, cat_rxn, unbinding_rxn]